In [1]:
%%javascript
if(IPython.tab_as_tab_everywhere)IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import os, sys, json
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import stats
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D,Dropout
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, f1_score


pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 200)

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [62]:
train = pd.read_csv('data/train.csv', encoding='ISO-8859-1').sample(frac=1)
test = pd.read_csv('data/test.csv', encoding='ISO-8859-1')

In [4]:
train[:10]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
6099,ID_28c6418a7,NaN,0,5,0,1,1,0,NaN,1,2,3,0,1,1,1,3,4,4,4,7,0.0,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1fe147787,2,2,0,4,yes,16,no,15.500000,0,0,0,1,0,0,0,0,0,2,2.000000,1,0,0,0,0,0,1,1,2,0,1,0,0,0,0,1,0,13,49,169,16,256,4,4.000000,1.000000,240.250000,169,4
5007,ID_1f69c7474,NaN,0,4,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,6,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,a1932650c,0,2,2,2,8,6,no,6.000000,0,0,1,0,0,0,0,0,0,1,2.000000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,87,36,7569,4,36,0,4.000000,64.000000,36.000000,7569,3
6906,ID_fdc985777,NaN,1,1,1,1,1,0,NaN,2,1,3,0,1,1,2,2,4,4,4,11,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1ce24a7b1,2,2,0,4,yes,6,no,8.500000,0,0,0,0,1,0,0,0,0,1,4.000000,0,0,0,0,1,0,0,1,2,0,0,1,0,0,0,1,0,28,121,784,16,36,4,16.000000,1.000000,72.250000,784,3
6071,ID_0c41ca724,NaN,0,4,0,1,1,0,NaN,0,2,2,0,2,2,0,4,4,4,4,10,NaN,4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,f15aa1b48,1,3,0,4,.33333334,6,no,8.333333,0,0,0,1,0,0,0,0,0,2,2.000000,1,0,0,0,0,0,0,1,3,0,1,0,0,0,0,1,0,20,100,400,16,36,1,4.000000,0.111111,69.444443,400,3
4662,ID_88ab33cbc,70000.0,0,5,0,1,1,0,NaN,0,1,1,1,1,2,1,2,3,3,3,17,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6597f05d4,1,2,0,3,.5,17,no,15.500000,0,0,0,0,0,0,0,0,1,2,1.500000,0,1,0,0,0,1,0,1,2,1,0,0,0,0,0,1,0,39,289,1521,9,289,1,2.250000,0.250000,240.250000,1521,4
2371,ID_0ca3dd2bd,NaN,0,4,0,1,1,1,1.0,0,1,1,0,2,2,0,3,3,3,3,11,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,c672764e7,0,3,1,3,.5,6,no,9.333333,0,0,0,0,1,0,0,0,0,3,1.000000,1,0,0,0,0,0,0,1,3,1,0,0,0,0,0,1,0,62,121,3844,9,36,0,1.000000,0.250000,87.111107,3844,4
6827,ID_9db89986c,40000.0,0,2,0,1,1,0,NaN,0,1,1,0,0,0,0,1,1,1,1,3,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,20801837a,0,1,1,1,8,3,no,3.000000,0,1,0,0,0,0,0,0,0,1,1.000000,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,88,9,7744,1,9,0,1.000000,64.000000,9.000000,7744,4
2616,ID_32983eb91,NaN,0,5,0,1,1,1,1.0,0,0,0,0,3,3,0,3,3

In [5]:
drop = ['Id','idhogar','elimbasu5','Target']
#probably useless features
#drop +=['planpri','paredother','coopele','estadocivil6','estadocivil7','dependency-yes','hogar_mayor','instlevel6','parentesco7','tipovivi1','tipovivi3','parentesco8','public','parentesco10','parentesco9','sanitario6','parentesco12','instlevel4','meaneduc-nan','SQBmeaned-nan','parentesco5','lugar2','parentesco11','techoentrepiso']
ids = ['Extreme Poverty','Moderate Poverty','Vulnerable Households','Non Vulnerable Households']

In [9]:
def build_structure():
	uniques = {}
	for col in train:
		if col in drop: continue
	
		num = pd.to_numeric(train[col], errors='coerce')
		null = np.isnan(num)
		column = np.nan_to_num(num.copy(),0)
		d_min = float(column.min())
		d_max = float(column.max())
		dif = d_max - d_min
	
	return {
		'uniques':uniques
	}
struct = build_structure()

In [10]:
def get_normalize(train,struct):
	df = pd.DataFrame()
	for col in train:
		if col in drop: continue
		num = pd.to_numeric(train[col], errors='coerce')
		null = np.isnan(num)
		df[col] = np.nan_to_num(num.copy(),0)
		
		if col in struct['uniques']:
			data_col = train[col].fillna('nan')
			for key in struct['uniques'][col]:
				df[col+ '-' + key] = (data_col == key).tolist()
		
		
	return df


In [11]:
df = get_normalize(train,struct)

In [55]:
target = train['Target']
one_hot = pd.get_dummies(train['Target'])

columns = len(df.columns)
train_percent = 0.9
train_num = int(len(train) * train_percent)
test_num = len(train) - train_num

train_x = df.head(train_num).as_matrix()
train_y = target.head(train_num).as_matrix()
train_y_hot = one_hot.head(train_num).as_matrix()


test_x = df.tail(test_num).as_matrix()
test_y = target.tail(test_num).as_matrix()
test_y_hot = one_hot.tail(test_num).as_matrix()


In [41]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [42]:
clf = RandomForestClassifier(n_estimators = 20)
clf = clf.fit(train_x, train_y)
guess = clf.predict(test_x)

print(f1_score(test_y, guess, average='macro'))
(test_y == guess).mean()

0.8766668279868036


0.9215481171548117

In [63]:
pre = np.array(clf.predict_proba(test_x))


total = 0
for i in range(4):
	total+=roc_auc_score(test_y_hot[:,i],pre[:,i])
total / 4

0.9798579220918535

In [64]:
test_df = get_normalize(test,struct)
output = pd.DataFrame()
output['Id'] = test['Id']
output['Target'] = clf.predict(test_df)
output.to_csv('decision.csv',index=False)

array([4, 4, 4, ..., 2, 2, 2])